In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
words = open('names.txt').read().splitlines()


In [4]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [5]:
# build the dataset
import random

blocksize = 3

def build_dataset(words):
    X = []
    Y = []
    for word in words:
        context = [0] * blocksize
        for ch in word + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]
    
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    return X,Y

random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))
[X, Y] = build_dataset(words[:n1])
[XV, YV] = build_dataset(words[n1:n2])
[XT, YT] = build_dataset(words[n2:])

In [6]:
# Two dimensional embedding
C = torch.randn((27, 10), requires_grad=True)
embedding_sum_size = C.shape[1] * blocksize

In [26]:
g = torch.Generator().manual_seed(2147483647) # for reproducibility
# https://pytorch.org/docs/stable/nn.init.html
n_hidden = 200
W1 = torch.randn((embedding_sum_size, n_hidden)) * ((5/3)/(embedding_sum_size**0.5))
# b1 = torch.randn(W1.shape[1]) * 0.01
W2 = torch.randn((W1.shape[1], 27)) * 0.01
b2 = torch.randn(W2.shape[1]) * 0

bngain = torch.ones((1, n_hidden))
bnbias = torch.zeros((1, n_hidden))
bnmean_running = torch.ones((1, n_hidden))
bnstd_running = torch.ones((1, n_hidden))

parameters = [C, W1, W2, b2, bngain, bnbias]

for p in parameters:
    p.requires_grad = True

In [29]:


@torch.no_grad()
def evaluate(emb, target):
    hpreact = emb.view(-1, embedding_sum_size) @ W1 # + b1
    hpreact = bngain * (hpreact - bnmean_running)/bnstd_running + bnbias
    h = torch.tanh(hpreact)
    # Output layer
    logits = h @ W2 + b2
    
    # Cross entropy loss
    anll = F.cross_entropy(logits, target)
    return anll

# I HAVE NO IDEA WHY I HAVE TO PASS THESE VARIABLES IN
def forward(emb, target, bnmean_running, bnstd_running):
    # Linear Layer       
    hpreact = emb.view(-1, embedding_sum_size) @ W1 # + b1

    # Batch normal layer
    bnmeani = hpreact.mean(0, keepdim=True)
    bnstdi = hpreact.std(0, keepdim=True)
    hpreact = bngain * (hpreact - bnmeani)/bnstdi + bnbias

    # Non linearity layer
    h = torch.tanh(hpreact)

    # Output linear layer
    logits = h @ W2 + b2
    
    # Cross entropy loss
    anll = F.cross_entropy(logits, target)

    # This should actually use https://en.wikipedia.org/wiki/Algorithms_for_calculating_variance
    # But really this is essentially what pytorch uses.
    with torch.no_grad():
        bnmean_running *= 0.999 
        bnmean_running += bnmeani * 0.001
        bnstd_running *= 0.999 
        bnstd_running += bnstdi * 0.001

    return anll

for _ in range(10000):
    ix = torch.randint(0, X.shape[0], (64, ))
    # Forward pass
    # Embedded values
    emb = C[X[ix]]
    
    # Cross entropy loss
    anll = forward(emb, Y[ix], bnmean_running, bnstd_running)

    for p in parameters:
        p.grad = None
        
    anll.backward()
    
    learning_rate = 0.05
    for p in parameters:
        p.data += -learning_rate * p.grad
        
print(f'Train: {evaluate(C[X], Y)} Validation: {evaluate(C[XV], YV)} Test: {evaluate(C[XT], YT)}')

Train: 2.1569712162017822 Validation: 2.1812727451324463 Test: 2.1791417598724365


In [187]:
# training split (for training), # dev/validation (for hyperparemeter tuning), # test split (for final evaluation)

In [195]:
for _ in range(20):
    out = []
    context = [0] * blocksize
    while True:
        emb = C[torch.tensor([context])]
        h = torch.tanh(emb.view(1, -1) @ W1 + b1)
        logits = h @ W2 + b2
        probs = F.softmax(logits, dim=1)
        ix = torch.multinomial(probs, num_samples=1).item()
        context = context[1:] + [ix]
        out.append(ix)
        if ix == 0:
            break
    print(''.join(itos[i] for i in out))

zeah.
buslaeseseuaushur.
raymeelyanah.
adie.
mulem.
issimarlandrostae.
malon.
mashet.
matir.
nana.
ala.
sily.
jona.
jyon.
kasharri.
tro.
zoler.
scee.
jasha.
gunta.
